In [ ]:
import re


def find_tax_fee(text):

    key_value_separator = r"(?:\s*(?:-|:)?\s*)"
    numeric_amount_1 = r"(?:\$\s*\d{1,3}(?:,\d{3})*(?:\.\d{2})?)"
    numeric_amount_2 = r"(?:\$\s*\d{1,6}(?:\.\d{2})?)"
    numeric_amount_3 = r"(?:\$?\s*\d{1,3}(?:,\d{3})+(?:\.\d{2})?)"

    numeric_amount = rf"(?:{numeric_amount_1}|{numeric_amount_2}|{numeric_amount_3})"
    explicit_numeric_amount = rf"(?:{numeric_amount_1}|{numeric_amount_2})"

    prefix1 = r"(?:\b[A-Za-z]+\b\s)?tax(?:es)?"
    tax_identifier = r"(?:excise|duty|stamp(?:s)?|city|county|state|district|township|municipality)"
    prefix2 = rf"(?:{tax_identifier}(?:\s\b[A-Za-z]+\b)?)"

    prefix = rf"(?:{prefix1}|{prefix2})"

    valid_pattern1 = rf"(?:{prefix1}{key_value_separator}\s*{numeric_amount})"
    valid_pattern2 = rf"(?:{tax_identifier}{key_value_separator}\s*{numeric_amount})"
    valid_pattern3 = rf"(?:{prefix2}{key_value_separator}\s*{explicit_numeric_amount})"

    valid_pattern = rf"(?:{valid_pattern1}|{valid_pattern2}|{valid_pattern3})"
    matches = re.findall(valid_pattern, text, re.IGNORECASE | re.DOTALL)

    results = []
    for match in matches:
        results.append((match))

    return results

def parse_dollar_amount(s):
    if not isinstance(s, str):
        return None

    cleaned = s.replace('$', '').replace(',', '').replace('(', '').replace(')', '').strip()
    print(cleaned)

    try:
        return float(cleaned)
    except ValueError:
        return None


def find_sales_amount(text):

  cardinal_1 = r"(?:one|two|three|four|five|six|seven|eight|nine)"
  cardinal_2 = r"(?:ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen)"
  cardinal_3 = r"(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)"

  compound = rf"(?:{cardinal_3}[-\s]+{cardinal_1})"
  cardinal = rf"(?:{compound}|{cardinal_3}|{cardinal_2}|{cardinal_1})"

  hundreds = rf"(?:{cardinal_1}\s+)?hundred(?:\s+{cardinal})?"
  group1 = rf"(?:{hundreds}|{cardinal})"

  thousands = rf"(?:{group1}\s+)?thousand(?:\s+{group1})?"
  group2 = rf"(?:{thousands}|{hundreds}|{cardinal})"

  millions = rf"(?:{cardinal}\s+)?million(?:\s+{group2})?"

  amount_in_words = rf"(?:{millions}|{thousands}|{hundreds}|{cardinal})"
  fractional_part = r"\s*and\s+(?:NO|00|\d{2})/100(?:th)?"

  full_amount_in_words = rf"(?:{amount_in_words}{fractional_part})"


  numeric_amount_without_parentheses = r"\$\s*\d{1,3}(?:\s*(?:,)?\s*\d{3})*(?:\.\d{2})?"
  numeric_amount_in_parentheses = rf"\(\s*{numeric_amount_without_parentheses}\s*\)"

  numeric_amount = rf"(?:{numeric_amount_without_parentheses}|{numeric_amount_in_parentheses})"

  valid_pattern1 = rf"(?:{full_amount_in_words}\s+dollars\s+{numeric_amount})"
  valid_pattern2 = rf"(?:{full_amount_in_words}\s+{numeric_amount}\s+dollars)"

  valid_patterns = rf"(?:{valid_pattern1}|{valid_pattern2})"

  plural = r"(?:\(s\))"
  other_valid_prefixes = rf"(?:sum{plural}?|amount{plural}?|price)"
  key_val_sep = r"(?:-|:|\s*)"



  full_pattern1 = rf"(?:consideration{key_val_sep}?(?:.){{0,200}}\s+{valid_patterns})"
  full_pattern2 = rf"(?:consideration{key_val_sep}?(?:.){{1,150}}\s+{numeric_amount})"
  full_pattern3 = rf"(?:{other_valid_prefixes}(?:.){{1,100}}\s+{valid_patterns})"
  full_pattern4 = rf"(?:{other_valid_prefixes}(?:.){{1,100}}\s+{numeric_amount})"


  full_pattern = rf"(?:{full_pattern1}|{full_pattern2}|{full_pattern3}|{full_pattern4})"

  matches = re.findall(full_pattern , text, re.IGNORECASE | re.DOTALL)
  amount = []

  for m in matches:
    a = re.search(numeric_amount, m)
    a = parse_dollar_amount(a.group())

    if a is not None and a > 10000:
      amount.append(a)

  return max(amount) if amount else None

text =  "143001 NUMBER OF IDS: 1 PARTY 1: FREDERICK J MEISS JR CONSIDERATION/SECURED AMT: $375,000.00 FEES / TAXES: Recording Fee $86.75 "
matches = find_sales_amount(text)
print(matches)